In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import psycopg2

In [2]:
# Make a reference to the books.csv file path
csv_path = "Data/annual_conc_by_monitor_2019.csv"

# Import the books.csv file as a DataFrame
air_df = pd.read_csv(csv_path, encoding="utf-8")
air_df.head()

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,75th Percentile,50th Percentile,10th Percentile,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
0,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,1 HOUR,...,0.054,0.045,0.029,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2/18/2020
1,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2/18/2020
2,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2/18/2020
3,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.050,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2/18/2020
4,1,3,10,88101,1,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,24 HOUR,...,9.500,7.300,3.800,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",3/10/2020


In [3]:
# Remove unecessary columns from the DataFrame and save the new DataFrame
base_df = air_df[["State Name", "City Name", "Latitude", "Longitude", "Parameter Name", "Arithmetic Mean", "Pollutant Standard"]]
base_df.head()

,State Name,City Name,Latitude,Longitude,Parameter Name,Arithmetic Mean,Pollutant Standard
0,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.044808,Ozone 1-hour 1979
1,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 1997
2,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 2008
3,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.040489,Ozone 8-hour 2015
4,Alabama,Fairhope,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 24-hour 2006


In [4]:
#Rename columns
base_df = base_df.rename(columns={
    "State Name": "State", 
    "City Name": "City",
    "Parameter Name": "Pollutant",
    "Arithmetic Mean": "Average"
})

#Set index to....city?
base_df.set_index("City", inplace=True)

base_df.head()

,State,Latitude,Longitude,Pollutant,Average,Pollutant Standard
City,,,,,,
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.044808,Ozone 1-hour 1979
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 1997
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 2008
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.040489,Ozone 8-hour 2015
Fairhope,Alabama,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 24-hour 2006


## OZONE DATA:

In [5]:
# Ozone df
ozone = base_df.loc[base_df['Pollutant'] == 'Ozone']
ozone.head()

,State,Latitude,Longitude,Pollutant,Average,Pollutant Standard
City,,,,,,
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.044808,Ozone 1-hour 1979
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 1997
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 2008
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.040489,Ozone 8-hour 2015
Muscle Shoals,Alabama,34.762619,-87.638097,Ozone,0.043192,Ozone 1-hour 1979


In [6]:
# Choosing to use only one pollutant standard (I chose the most recent which is Ozone 8hr 2015)
# I did this so each city (or coordinates) will have one of each test data
ozone = ozone.loc[ozone['Pollutant Standard'] == 'Ozone 8-hour 2015']
ozone.head()

,State,Latitude,Longitude,Pollutant,Average,Pollutant Standard
City,,,,,,
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.040489,Ozone 8-hour 2015
Muscle Shoals,Alabama,34.762619,-87.638097,Ozone,0.038221,Ozone 8-hour 2015
Crossville,Alabama,34.289001,-85.970065,Ozone,0.044582,Ozone 8-hour 2015
Wetumpka,Alabama,32.535680,-86.255193,Ozone,0.040902,Ozone 8-hour 2015
Southside,Alabama,33.904039,-86.053867,Ozone,0.044016,Ozone 8-hour 2015


In [7]:
# Search for duplicates
ozone.duplicated()

City
Fairhope         False
Muscle Shoals    False
Crossville       False
Wetumpka         False
Southside        False
                 ...  
NaN              False
Bayamon          False
Catano           False
Mayaguez         False
NaN              False
Length: 1304, dtype: bool

In [8]:
# Search for nan values
ozone.isna().any()

State                 False
Latitude              False
Longitude             False
Pollutant             False
Average               False
Pollutant Standard    False
dtype: bool

## PM.2.5 DATA

In [9]:
# working off the base
base_df.head()

,State,Latitude,Longitude,Pollutant,Average,Pollutant Standard
City,,,,,,
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.044808,Ozone 1-hour 1979
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 1997
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.040390,Ozone 8-Hour 2008
Fairhope,Alabama,30.497478,-87.880258,Ozone,0.040489,Ozone 8-hour 2015
Fairhope,Alabama,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 24-hour 2006


In [10]:
# Ozone df
pm = base_df.loc[base_df['Pollutant'] == 'PM2.5 - Local Conditions']
pm.head()

,State,Latitude,Longitude,Pollutant,Average,Pollutant Standard
City,,,,,,
Fairhope,Alabama,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 24-hour 2006
Fairhope,Alabama,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 24-hour 2012
Fairhope,Alabama,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 Annual 2006
Fairhope,Alabama,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 Annual 2012
Ashland,Alabama,33.284928,-85.803608,PM2.5 - Local Conditions,7.550467,PM25 24-hour 2006


In [11]:
# Choosing to use only one pollutant standard (I chose the most recent which is PM2.5 Annual 2012)
# I did this so each city (or coordinates) will have one of each test data
pm = pm.loc[pm['Pollutant Standard'] == 'PM25 Annual 2012']
pm.head()

,State,Latitude,Longitude,Pollutant,Average,Pollutant Standard
City,,,,,,
Fairhope,Alabama,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402,PM25 Annual 2012
Ashland,Alabama,33.284928,-85.803608,PM2.5 - Local Conditions,7.550467,PM25 Annual 2012
Muscle Shoals,Alabama,34.762619,-87.638097,PM2.5 - Local Conditions,7.213235,PM25 Annual 2012
Crossville,Alabama,34.288567,-85.969858,PM2.5 - Local Conditions,7.550000,PM25 Annual 2012
Gadsden,Alabama,33.991494,-85.992647,PM2.5 - Local Conditions,8.410526,PM25 Annual 2012


In [12]:
# Search for duplicates
pm.duplicated()

City
Fairhope            False
Ashland             False
Muscle Shoals       False
Crossville          False
Gadsden             False
                    ...  
Guaynabo            False
Guaynabo            False
Ponce               False
NaN                 False
Charlotte Amalie    False
Length: 1565, dtype: bool

In [13]:
# Search for nan values
pm.isna().any()

State                 False
Latitude              False
Longitude             False
Pollutant             False
Average               False
Pollutant Standard    False
dtype: bool

## CSV COPIES

In [ ]:
# csv files of each df.. just in case?
base_df.to_csv('baseDFcsv.csv', index=False, header=True)

In [ ]:
ozone.to_csv('ozoneDFcsv.csv', index=False, header=True)

In [ ]:
pm.to_csv('pmDFcsv.csv', index=False, header=True)

## SQLITE DATABASE

In [14]:
from sqlalchemy import create_engine

In [15]:
# Connect to the database
engine = create_engine('sqlite:///aqi.sqlite', echo=False)
# I also created a db file:
# engine = create_engine('sqlite:///aqi.db', echo=False)
sqlite_connection = engine.connect()

In [ ]:
# Adding the base DF (I think this would give it its own table?)
base_df.to_sql('base', con=engine)

In [ ]:
# Adding the ozone DF
ozone.to_sql('ozone', con=engine)

In [ ]:
# Adding the pm DF
pm.to_sql('pm', con=engine)

In [16]:
# Confirming tables are in the DB
engine.execute("SELECT * FROM base").fetchall()

[('Fairhope', 'Alabama', 30.497478000000005, -87.880258, 'Ozone', 0.044808, 'Ozone 1-hour 1979'),
 ('Fairhope', 'Alabama', 30.497478000000005, -87.880258, 'Ozone', 0.040389999999999995, 'Ozone 8-Hour 1997'),
 ('Fairhope', 'Alabama', 30.497478000000005, -87.880258, 'Ozone', 0.040389999999999995, 'Ozone 8-Hour 2008'),
 ('Fairhope', 'Alabama', 30.497478000000005, -87.880258, 'Ozone', 0.040489, 'Ozone 8-hour 2015'),
 ('Fairhope', 'Alabama', 30.497478000000005, -87.880258, 'PM2.5 - Local Conditions', 7.551402, 'PM25 24-hour 2006'),
 ('Fairhope', 'Alabama', 30.497478000000005, -87.880258, 'PM2.5 - Local Conditions', 7.551402, 'PM25 24-hour 2012'),
 ('Fairhope', 'Alabama', 30.497478000000005, -87.880258, 'PM2.5 - Local Conditions', 7.551402, 'PM25 Annual 2006'),
 ('Fairhope', 'Alabama', 30.497478000000005, -87.880258, 'PM2.5 - Local Conditions', 7.551402, 'PM25 Annual 2012'),
 ('Ashland', 'Alabama', 33.284928, -85.803608, 'PM2.5 - Local Conditions', 7.550467, 'PM25 24-hour 2006'),
 ('Ashland'

In [17]:
# ozone check
engine.execute("SELECT * FROM ozone").fetchall()

[('Fairhope', 'Alabama', 30.497478000000005, -87.880258, 'Ozone', 0.040489, 'Ozone 8-hour 2015'),
 ('Muscle Shoals', 'Alabama', 34.762619, -87.638097, 'Ozone', 0.038221, 'Ozone 8-hour 2015'),
 ('Crossville', 'Alabama', 34.289001, -85.970065, 'Ozone', 0.044582, 'Ozone 8-hour 2015'),
 ('Wetumpka', 'Alabama', 32.53568, -86.255193, 'Ozone', 0.040902, 'Ozone 8-hour 2015'),
 ('Southside', 'Alabama', 33.904039000000004, -86.05386700000001, 'Ozone', 0.044016, 'Ozone 8-hour 2015'),
 ('Dothan', 'Alabama', 31.188933000000002, -85.42309399999999, 'Ozone', 0.037248, 'Ozone 8-hour 2015'),
 ('Birmingham', 'Alabama', 33.553056, -86.815, 'Ozone', 0.039671, 'Ozone 8-hour 2015'),
 ('Fairfield', 'Alabama', 33.485556, -86.915, 'Ozone', 0.044638, 'Ozone 8-hour 2015'),
 (None, 'Alabama', 33.331111, -87.00361099999999, 'Ozone', 0.043936, 'Ozone 8-hour 2015'),
 ('Leeds', 'Alabama', 33.545278, -86.54916700000001, 'Ozone', 0.043135, 'Ozone 8-hour 2015'),
 (None, 'Alabama', 33.801667, -86.9425, 'Ozone', 0.043668,

In [18]:
# pm check
engine.execute("SELECT * FROM pm").fetchall()

[('Fairhope', 'Alabama', 30.497478000000005, -87.880258, 'PM2.5 - Local Conditions', 7.551402, 'PM25 Annual 2012'),
 ('Ashland', 'Alabama', 33.284928, -85.803608, 'PM2.5 - Local Conditions', 7.550467, 'PM25 Annual 2012'),
 ('Muscle Shoals', 'Alabama', 34.762619, -87.638097, 'PM2.5 - Local Conditions', 7.213235000000001, 'PM25 Annual 2012'),
 ('Crossville', 'Alabama', 34.288567, -85.969858, 'PM2.5 - Local Conditions', 7.55, 'PM25 Annual 2012'),
 ('Gadsden', 'Alabama', 33.991494, -85.99264699999999, 'PM2.5 - Local Conditions', 8.410525999999999, 'PM25 Annual 2012'),
 ('Dothan', 'Alabama', 31.224783000000002, -85.390789, 'PM2.5 - Local Conditions', 8.115652, 'PM25 Annual 2012'),
 ('Birmingham', 'Alabama', 33.553056, -86.815, 'PM2.5 - Local Conditions', 10.310833, 'PM25 Annual 2012'),
 ('Birmingham', 'Alabama', 33.553056, -86.815, 'PM2.5 - Local Conditions', 10.1, 'PM25 Annual 2012'),
 ('Birmingham', 'Alabama', 33.553056, -86.815, 'PM2.5 - Local Conditions', 10.552222, 'PM25 Annual 2012'),

In [19]:
# All the table names in the sqlite file (aqi.sqlite)
engine.table_names()

['aqi', 'base', 'ozone', 'pm']